<div align="left" style="position: relative;">
<img src="https://image.rocketpunch.com/company/131793/spark-x_logo_1633693895.png?s=400x400&t=inside" align="left" width="30%" style="margin: 20px 60px 50px 20px;">
<h1><strong>두산 로봇 교육 자료</strong></h1>
<p align="left">
	<em><code>❯ written by <strong>명학, 재승</strong></code></em>
</p>
<p align="left">
</p>
<p align="left">
	<!-- default option, no dependency badges. -->
</p>
</div>
<br clear="right">

##  Table of Contents

1. Get Ready
2. movej & movel 
3. amovej & amovel
4. get_current_posx()
5. Mode Change
6. Grip, Release (Digital IO, Modbus)
7. Force Control
8. Move Periodic

---

### 1. Get Ready
두산 로봇 ROS2 패키지를 설치한다. (설치 링크: https://boundless-binder-063.notion.site/ROS-and-DR-1389786e552e800480e8d88cfb5f2fb5)

다음 명령어로 m0609를 실행한다.
* real  
  `ros2 launch dsr_bringup2 dsr_bringup2_rviz.launch.py mode:=real host:=192.168.1.100 port:=12345 model:=m0609`
* virtual  
  `ros2 launch dsr_bringup2 dsr_bringup2_rviz.launch.py mode:=virtual host:=127.0.0.1 port:=12345 model:=m0609`

터미널을 새로 열고 ros2_ws로 이동  
`source install/setup.bash`를 하고 code . 으로 vscode를 실행한다.  
그 다음 *.ipynb(주피터 노트북 파일)을 만들어 실습을 시작한다.

In [ ]:
#필요한 모듈 불러오기 및 로봇 기본 설정
import sys, os
sys.path.append(os.path.expanduser('~/ros2_ws/install/common2/lib/common2/imp'))
import DR_init

ROBOT_ID = "dsr01"
ROBOT_MODEL = "m0609"
VELOCITY, ACC = 60, 60

DR_init.__dsr__id = ROBOT_ID
DR_init.__dsr__model = ROBOT_MODEL

In [2]:
#로봇 노드 실행
import rclpy
rclpy.init()
node = rclpy.create_node("rokey_simple_move", namespace=ROBOT_ID)

DR_init.__dsr__node = node

---

### 2. movej & movel

#### movej( [j1, j2, j3 ,4, j5, j6], vel, acc)  
> 각 joint에 대해 각도를 지정하고 적절한 vel(속도)과 acc(가속도)를 설정하여 로봇을 움직인다.  
#### movel( [x, y, z, rx, ry, rz], vel, acc)  
> 로봇의 엔드포인트(그리퍼나 joint6)의 좌표를 지정하고 적절한 적절한 vel(속도)과 acc(가속도)를 설정하여 로봇을 움직인다.  
#### movej vs movel  
> movej는 joint의 각도를 직접적으로 제어하여 joint의 각도 제한 내에서는 특이점이 발생하지 않는다.  
> movel은 목표 좌표에 대해서 역기구학으로 joint 제어를 계산하므로 특이점이 발생할 수 있다.  
> movel의 좌표는 역기구학의 특성상 여러 개의 해가 존재할 수 있다.(같은 자세라도 좌표가 다를 수 있다.) 

<p align="left">
  <img src="movej.png" width="20%" />
  <img src="movel.png" width="20%" />
</p>


In [3]:
#movej와 movel 함수 가져오기
try:
    from DSR_ROBOT2 import (
        movej,
        movel,
    )
except ImportError as e:
    print(f"Error importing DSR_ROBOT2 : {e}")

_robot_id =dsr01
_robot_model =m0609
_srv_name_prefix =
_topic_name_prefix =


In [12]:

VELOCITY, ACC = 60, 60

# Initial Pose : movej로 이동
init_pose_j = [0, 0, 0, 0, 0, 0]
movej(init_pose_j, vel=VELOCITY, acc=ACC)

0

In [ ]:
# Home Pose : movej로 이동
home_pose_j = [0, 0, 90, 0, 90, 0]
movej(home_pose_j, vel=VELOCITY, acc=ACC)

0

In [29]:
# Home Pose : movel로 이동
home_pose_l = [350, 350, 450, 0, 180, 0]
movel(home_pose_l, vel=VELOCITY, acc=ACC)


0

In [30]:
# Initial Pose : movel로 이동
init_pose_l = [400, 300, 500, 0, 180, 0]
movel(init_pose_l, vel=VELOCITY, acc=ACC)

0

In [31]:
#home/init pose로 가는 함수는 movej로 정의하는 것이 더 유리하다.(더 직관적 표현, 특이점 회피, 빠른 이동 등)
def go_home():
    home_pose_j = [0, 0, 90, 0, 90, 0]
    movej(home_pose_j, vel=VELOCITY, acc=ACC)

def go_init():
    init_pose_j = [0, 0, 0, 0, 0, 0]
    movej(init_pose_j, vel=VELOCITY, acc=ACC)

---

### 3. amovej & amovel

#### amovej( [j1, j2, j3 ,j4, j5, j6], vel, acc)  
> 각 joint에 대해 각도를 지정하고 적절한 vel(속도)과 acc(가속도)를 설정하여 로봇을 **비동기로** 움직인다.  
#### amovel( [x, y, z, rx, ry, rz], vel, acc)  
> 로봇의 엔드포인트(그리퍼나 joint6)의 좌표를 지정하고 적절한 적절한 vel(속도)과 acc(가속도)를 설정하여 로봇을 **비동기로** 움직인다.  
#### 동기 실행(sync) vs 비동기 실행(async)  
> 동기 실행은 네트워크 통신이나 다른 기기를 제어할 때 그 작업이 끝나기를 기다리는 것  
> 비동기 실행은 네트워크 통신이나 다른 기기를 제어할 때 그 작업을 실행한 뒤 곧이어 다른 코드를 실행하는 것



---

In [32]:
#movej와 movel 함수 가져오기
try:
    from DSR_ROBOT2 import (
        amovej,
        amovel,
    )
except ImportError as e:
    print(f"Error importing DSR_ROBOT2 : {e}")

In [37]:
import time

# 동기 방식
go_home()
print("동기 movej 호출 전 시각:", time.time())
movej([0, 0, 0, 0, 0, 0], vel=60, acc=60)
print("동기 movej 호출 후 시각:", time.time())
print("동기 movej 명령 완료 후 실행.\n")

# 비동기 방식 예제
go_home()
print("비동기 amovej 호출 전 시각:", time.time())
amovej([0, 0, 0, 0, 0, 0], vel=60, acc=60)
print("비동기 amovej 호출 후 시각:", time.time())
print("비동기 amovej 명령 호출 후 곧 바로 다음 코드 실행")

동기 movej 호출 전 시각: 1743412299.8661432
동기 movej 호출 후 시각: 1743412302.6784039
동기 movej 명령 완료 후 실행.

비동기 amovej 호출 전 시각: 1743412305.498741
비동기 amovej 호출 후 시각: 1743412305.5017266
비동기 amovej 명령 호출 후 곧 바로 다음 코드 실행


### 4. get_current_posx()

#### conv_posx, sol = get_current_posx()
* conv_posx : 현재 엔드이펙터의 위치  
* sol : 엔드이펙터를 표현하는 좌표가 여러 개일 수 있는데 그 중 몇번째 좌표를 선택했는지

In [38]:
#movej와 movel 함수 가져오기
try:
    from DSR_ROBOT2 import (
        get_current_posx
    )
except ImportError as e:
    print(f"Error importing DSR_ROBOT2 : {e}")

In [ ]:
movej([0, 0, 90, 0, 90, 0], vel=VELOCITY, acc=ACC)
current_pos = get_current_posx()[0] # [0]으로 하는 이유는 현재의 posx만 추출해내기 위함
print(current_pos)
current_pos[2] += 200 # z축으로 200만큼 위로 이동
movel(current_pos, vel=VELOCITY, acc=ACC)

[559.000, 34.500, 651.500, 45.000, 180.000, 45.000]


0

### 5. Mode Change

#### SetRobotMode
로봇의 모드를 설정하기 위해 서비스를 호출할 수 있다. (virtual mode에서는 확인하기 힘듬)
* mode=0 : 로봇 수동 제어(파란불)
* mode=1 : 로봇 자동(원격) 제어(하얀불)

In [ ]:
# 로봇 수동 제어 모드로 설정
!ros2 service call /dsr01/system/set_robot_mode dsr_msgs2/srv/SetRobotMode "{robot_mode: 0}"

# 로봇 자동 제어 모드로 설정
!ros2 service call /dsr01/system/set_robot_mode dsr_msgs2/srv/SetRobotMode "{robot_mode: 1}"

In [ ]:
import rclpy
from dsr_msgs2.srv import SetRobotMode

def send_request(node, mode=0):
    # 서비스 요청 메시지 생성 및 필드 설정
    request = SetRobotMode.Request()
    request.robot_mode = mode
    future = node.client.call_async(request)
    rclpy.spin_until_future_complete(node, future)
    return future.result()


node = rclpy.create_node('set_robot_mode_client')

# 서비스 클라이언트 생성
node.client = node.create_client(SetRobotMode, "/dsr01/system/set_robot_mode")

# 서비스가 준비될 때까지 대기
while not node.client.wait_for_service(timeout_sec=1.0):
    print("Waiting for service...")

In [ ]:
# 서비스 요청 보내기 (예시로 mode=0을 전달)
response = send_request(node, mode=0)
print("Service response:", response)

In [ ]:
# 로봇 모드 확인하는 서비스 호출
!ros2 service call /dsr01/system/get_robot_mode dsr_msgs2/srv/GetRobotMode

In [ ]:
# 노드 종료 및 rclpy 종료
node.destroy_node()
rclpy.shutdown()

### 6. Grip, Release (Digital IO, Modbus)

![](rg2.jpeg)

* 그리퍼 제어(virtual mode에서는 확인하기 힘듬)  
  6-1. 그리퍼를 Digital IO로 연결했을 때  
  6-2.  Modbus로 연결했을 때

#### 6-1 Digital IO

로봇의 제어기에 1번, 2번 IO에 핀을 꽂아서 연결

In [ ]:
ON, OFF = 1, 0

# 컨트롤러의 디지털 입력 신호 = 그리퍼의 디지털 출력 신호
# 컨트롤러의 디지털 출력 신호 = 그리퍼의 디지털 입력 신호
try:
    from DSR_ROBOT2 import (
        set_digital_output, # 컨트롤러에서 신호를 내보내어 그리퍼에 입력
        get_digital_input, # 컨트롤러의 입력 신호 상태를 불러옴
        set_tool,
        set_tcp,
        wait,
    )
except ImportError as e:
    print(f"Error importing DSR_ROBOT2 : {e}")


In [ ]:
def wait_digital_input(sig_num): # sig_num에 해당하는 핀이 ON(1)이 될 때까지 기다리기
    while not get_digital_input(sig_num):
        wait(0.5)
        print("Wait for digital input")
        pass

def release(): # 2번 핀을 ON, 1번 핀을 OFF 하면 그리퍼가 열림
    set_digital_output(1, OFF)
    set_digital_output(2, ON)
    wait_digital_input(2) # 그리퍼의 동작이 끝날 때까지 다른 동작을 수행하지 못하게 하는 역할(작업 완료 시까지 대기)

def grip(): # 1번 핀을 ON, 2번 핀을 OFF 하면 그리퍼가 열림
    release()
    set_digital_output(1, ON)
    set_digital_output(2, OFF)
    wait_digital_input(1)

set_tool("Tool Weight_2FG")
set_tcp("2FG_TCP")

In [ ]:
grip()
print("grip")
release()
print("release")

#### 6-2. Modbus

로봇을 랜선으로 연결하고 Modbus 프로토콜로 통신  
* **Modbus란?**  
그리퍼를 TCP/IP를 통해 데이터를 송수신할 수 있게 지원하는 프로토콜 중 하나이다.  
그리퍼를 Modbus로 제어하면 더 자유롭게 제어할 수 있다.  

~/ros2_ws/src/doosan-robot2/common2/imp 아래에 onrobot.py가 존재해야 함  
다운받기 : [onrobot-gripper-modbus](https://github.com/takuya-ki/onrobot-rg)

In [ ]:
#그리퍼 모듈 불러오기
from onrobot import RG

#그리퍼 관련 설정
GRIPPER_NAME = "rg2"
TOOLCHARGER_IP = "192.168.1.1"
TOOLCHARGER_PORT = "502"

#그리퍼 객체 생성
gripper = RG(GRIPPER_NAME, TOOLCHARGER_IP, TOOLCHARGER_PORT)

In [ ]:
#그리퍼 열기
gripper.open_gripper()

#그리퍼의 상태를 체크하고 동작 중이면 wait(동작이 끝날 때까지 기다리기)
while gripper.get_status()[0]:
	time.sleep(0.5)

#그리퍼 닫기
gripper.close_gripper()
while gripper.get_status()[0]:
	time.sleep(0.5)


#그리퍼의 width를 직접 조정, 10N의 힘으로 조정
gripper.move_gripper(width_val=40, force_val=10)
while gripper.get_status()[0]:
	time.sleep(0.5)

#그리퍼의 현재 너비 출력
print(f'get_width_with_offset: {gripper.get_width_with_offset()}')

### 7. Force Control

#### 기초 개념
* 순응성 : 외부의 힘의 방향대로 움직이려는 특성  
  <!-- 값이 클수록 더 유연하게 움직임(순응성 값이 클 때 내가 로봇을 밀면 더 많이 밀림) -->
  용수철 원리처럼 설정된 강성에 따라 외력을 줄여주는 방향으로 순응하면서 모션 제어 수행

  설정하는 강성(stiffness, N/m) 값이 낮을 수록 외력에 부드럽게 반응

* 힘제어 : 외부의 힘을 감지 후 반력을 가하여 로봇을 제어  
![](force_control.png)

In [ ]:
try:
    from DSR_ROBOT2 import (
        release_compliance_ctrl,
        check_force_condition,
        task_compliance_ctrl,
        set_desired_force,
        DR_FC_MOD_REL,
        DR_AXIS_Z,
    )
except ImportError as e:
    print(f"Error importing DSR_ROBOT2 : {e}")

In [ ]:
go_home()

#stx : x, y, z, rx, ry, rz에 대한 강성 설정
task_compliance_ctrl(stx=[500, 500, 500, 100, 100, 100])

# x, y, z, rx, ry, rz
# fd: 원하는 힘 벡터 (Z축으로 -10N으로 밀기). fd가 음수이므로 아래 방향으로 10N의 힘으로 로봇 밀기
# dir: 힘이 적용될 방향 (Z축). 0: 해당 방향 순응 제어, 1: 해당 방향 힘 제어
# mod=DR_FC_MOD_REL: 상대 모드, 로봇 TCP 기준 방향
set_desired_force(fd=[0, 0, -10, 0, 0, 0], dir=[0, 0, 1, 0, 0, 0], mod=DR_FC_MOD_REL)

# Z축 방향에 5N 이상의 힘이 감지될 때까지 대기
# check_force_condition()이 True가 되면 루프 탈출
while not check_force_condition(DR_AXIS_Z, max=5):
    pass

# 힘 제어 해제, 이후 로봇은 다시 기본 모드로 복귀
release_compliance_ctrl()

### 8. Move Periodic
설정한 진폭만큼 sin함수를 따라 움직임  

진폭을 10으로 설정하면 처음위치 -> +10 위치 -> 처음위치 -> -10 위치 -> 처음위치를 왕복운동함 

move_periodic(amp, period, atime, repeat, ref): 동기 방식
<!-- #### amove_periodic(amp=[j1, j2, j3, j4, j5, j6], period, repeat) -->
amove_periodic(amp, period, atime, repeat, ref): 비동기 방식 

* amp(진폭) : 각 축 (x, y, z, rx, ry, rz)이 움직이는 폭의 정도
* period(주기) : 왕복 운동을 하는 주기
* atime(가감속시간): 주기모션의 시작과 끝의 가속 및 감속 시간, 입력된 가감속시간과 최대주기*1/4 중 큰 값이 적용됨. default 값은 0.0
* repeat(횟수) : 왕복 운동의 횟수. default 값은 1
* ref(기준 좌표계)
    - DR_BASE : base coordinate
    - DR_WORLD : world coordinate
    - DR_TOOL : tool coordinate (default)
    - user coordinate : 사용자 정의

![](movePeriodic.png)

In [ ]:
try:
    from DSR_ROBOT2 import (
        amove_periodic,
        DR_TOOL,
    )
except ImportError as e:
    print(f"Error importing DSR_ROBOT2 : {e}")

In [ ]:
go_home()
amove_periodic(amp=[30, 0, 0, 0, 0, 0], period=2.0, repeat=3)